In [1]:
#필요 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#한글 폰트 지정
import matplotlib
import matplotlib.font_manager as fm


font_path = "/System/Library/Fonts/Supplemental/AppleGothic.ttf"
fontprop = fm.FontProperties(fname=font_path)
matplotlib.rc('font', family=fontprop.get_name())

In [3]:
#데이터 경로 지정
order_data_file_path = "/Users/jangjuyeong/Documents/메타코드(spyder)/패션산업데이터분석_강의자료_메타코드M_구글샘플데이터_삭제_직전/Data/order_detail.csv"
customer_data_file_path = "/Users/jangjuyeong/Documents/메타코드(spyder)/패션산업데이터분석_강의자료_메타코드M_구글샘플데이터_삭제_직전/Data/customer_data.csv"

#데이터 불러오기
order_data = pd.read_csv(order_data_file_path)
customer_data = pd.read_csv(customer_data_file_path)

In [4]:
#주문일자 컬럼 생성
order_data["주문일자"] = order_data["주문일시"].str.slice(start=0, stop=10)
order_data["주문일자"] = pd.to_datetime(order_data["주문일자"], format="%Y-%m-%d")

마르코프 모형을 지정하기 위한 라벨링

In [5]:
order_data

,주문번호,고객번호,상품번호,컬러번호,사이즈,주문일시,정상가,판매가,주문수량,총주문금액,총주문금액_판매가,주문일자
0,ORD202401010002,CST2021091242907,GODAP24FW003425,GR,1,2024-01-01 23:20:02,206500,124073,1,206500,124073,2024-01-01
1,ORD202401010004,CST2020032411213,GODZN24SS005687,WT,6,2024-01-01 05:03:44,59100,48240,1,59100,48240,2024-01-01
2,ORD202401010005,CST2022083117975,GODZN24SS006196,BK,6,2024-01-01 15:17:23,64600,50789,2,129200,101578,2024-01-01
3,ORD202401010007,CST2020091521390,GODOR24SS007480,SY,1,2024-01-01 12:30:39,36200,19419,1,36200,19419,2024-01-01
4,ORD202401010010,CST202207159828,GODUC24SS009275,YW,2,2024-01-01 02:57:44,589100,354992,2,1178200,709984,2024-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
80439,ORD202408311739,CST202010317576,GODAP24SS004720,WT,3,2024-08-31 11:55:15,426600,319232,1,426600,319232,2024-08-31
80440,ORD202408311740,CST2022112940979,GODZR24FW006400,BK,1,2024-08-31 01:55:28,1189900,945906,2,2379800,1891812,2024-08-31
80441,ORD202408311745,CST2020072814354,GODZN24SS008240,GR,3,2024-08-31 02:08:11,29100,14770,3,87300,44310,2024-08-31
80442,ORD202408311746,CST2021122534627,GODHN24SS004759,YW,5,2024-08-31 09:39:43,758300,694630,1,758300,694630,2024-08-31


In [6]:
order_data["주문수량"] = order_data["주문수량"].astype(str)

In [7]:
order_data["할인율"] = 1 - (order_data["판매가"]/order_data["정상가"])

In [8]:
#할인수준
order_data.loc[(order_data["할인율"]>=0) & (order_data["할인율"]<0.3), "할인구분"] = "NORMAL"
order_data.loc[(order_data["할인율"]>=0.3) & (order_data["할인율"]<0.4), "할인구분"] = "SEASON OFF"
order_data.loc[(order_data["할인율"]>=0.4), "할인구분"] = "OUTLET"

In [9]:
order_data = order_data.dropna()

In [10]:
order_data

,주문번호,고객번호,상품번호,컬러번호,사이즈,주문일시,정상가,판매가,주문수량,총주문금액,총주문금액_판매가,주문일자,할인율,할인구분
0,ORD202401010002,CST2021091242907,GODAP24FW003425,GR,1,2024-01-01 23:20:02,206500,124073,1,206500,124073,2024-01-01,0.399162,SEASON OFF
1,ORD202401010004,CST2020032411213,GODZN24SS005687,WT,6,2024-01-01 05:03:44,59100,48240,1,59100,48240,2024-01-01,0.183756,NORMAL
2,ORD202401010005,CST2022083117975,GODZN24SS006196,BK,6,2024-01-01 15:17:23,64600,50789,2,129200,101578,2024-01-01,0.213793,NORMAL
3,ORD202401010007,CST2020091521390,GODOR24SS007480,SY,1,2024-01-01 12:30:39,36200,19419,1,36200,19419,2024-01-01,0.463564,OUTLET
4,ORD202401010010,CST202207159828,GODUC24SS009275,YW,2,2024-01-01 02:57:44,589100,354992,2,1178200,709984,2024-01-01,0.397399,SEASON OFF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80439,ORD202408311739,CST202010317576,GODAP24SS004720,WT,3,2024-08-31 11:55:15,426600,319232,1,426600,319232,2024-08-31,0.251683,NORMAL
80440,ORD202408311740,CST2022112940979,GODZR24FW006400,BK,1,2024-08-31 01:55:28,1189900,945906,2,2379800,1891812,2024-08-31,0.205054,NORMAL
80441,ORD202408311745,CST2020072814354,GODZN24SS008240,GR,3,2024-08-31 02:08:11,29100,14770,3,87300,44310,2024-08-31,0.492440,OUTLET
80442,ORD202408311746,CST2021122534627,GODHN24SS004759,YW,5,2024-08-31 09:39:43,758300,694630,1,758300,694630,2024-08-31,0.083964,NORMAL


은닉 마르코프 모형 시퀀스 만들기
-고객별로 주문한 히스토리 리스트

In [11]:
order_data.groupby('고객번호')

In [14]:
customer_data_sequence = []
for customer_ID, group in order_data.groupby('고객번호'):
    sales_level_sequence = group['할인구분'].tolist()
    purchase_amount_sequence = group['주문수량'].tolist()
    customer_data_sequence.append((customer_ID, sales_level_sequence, purchase_amount_sequence))

In [15]:
customer_data_sequence

[('CST202001020725', ['OUTLET'], ['2']),
 ('CST2020010212955', ['NORMAL', 'SEASON OFF'], ['2', '1']),
 ('CST2020010213687', ['NORMAL', 'SEASON OFF'], ['1', '1']),
 ('CST2020010213909',
  ['SEASON OFF', 'SEASON OFF', 'SEASON OFF'],
  ['2', '3', '2']),
 ('CST2020010215699', ['SEASON OFF', 'OUTLET', 'NORMAL'], ['1', '3', '1']),
 ('CST2020010217369', ['OUTLET', 'SEASON OFF'], ['1', '2']),
 ('CST2020010219669', ['NORMAL'], ['2']),
 ('CST2020010221534', ['NORMAL', 'NORMAL'], ['1', '2']),
 ('CST2020010222259', ['SEASON OFF'], ['3']),
 ('CST2020010223171',
  ['NORMAL', 'SEASON OFF', 'SEASON OFF', 'NORMAL', 'OUTLET', 'NORMAL'],
  ['2', '2', '1', '3', '3', '3']),
 ('CST2020010224862', ['SEASON OFF', 'NORMAL', 'NORMAL'], ['3', '1', '1']),
 ('CST2020010225277', ['NORMAL'], ['1']),
 ('CST202001022664', ['OUTLET', 'NORMAL', 'SEASON OFF'], ['2', '3', '1']),
 ('CST2020010228445', ['NORMAL'], ['1']),
 ('CST2020010231777', ['NORMAL'], ['1']),
 ('CST2020010232011', ['OUTLET'], ['3']),
 ('CST2020010233841

In [16]:
len(customer_data_sequence)

31766

은닉 마르코프 모형 실행하기

In [17]:
import numpy as np

hidden_states = ["NORMAL", "SEASON OFF", "OUTLET"]
observed_states = ["1", "2", "3"]

In [ ]:
#상태 전이 행렬 Transition probability 함수
def calculate_transition_matrix(customer_data, hidden_states):
    transition_counts = {state: {next_state: 0 for next_state in hidden_states} \
                         for state in hidden_states}
    
    for _, hidden_states_list, _ in customer_data: 
        for i in range(len(hidden_states_list) - 1): 
            current_state = hidden_states_list[i] 
            next_state = hidden_states_list[i + 1]
            transition_counts[current_state][next_state] += 1 

    transition_matrix = {state: {next_state: count \
                                 / sum(transitions.values()) \
                                 for next_state, count in transitions.items()} \
                         for state, transitions in transition_counts.items()}
    
    return transition_matrix

In [20]:
# 출력 행렬 emission(output) probability 함수 만들기
def calculate_emission_matrix(customer_data, hidden_states, observed_states):
    emission_counts = {state: {obs_state: 0 for obs_state in observed_states} \
                       for state in hidden_states}
    for _, hidden_states_list, observed_states_list in customer_data:
        for obs_state, hidden_state in zip(observed_states_list, \
                                           hidden_states_list):
            emission_counts[hidden_state][obs_state] += 1

    emission_matrix = {state: {obs_state: count / sum(emissions.values()) \
                               for obs_state, count in emissions.items()} \
                       for state, emissions in emission_counts.items()}
    
    return emission_matrix

In [21]:
# transition 계산하기
transition_matrix = calculate_transition_matrix(customer_data_sequence, \
                                                hidden_states)
print("Transition 결과:")
for state, transitions in transition_matrix.items():
    print(state + ": ", transitions)

# emission 계산하기
emission_matrix = calculate_emission_matrix(customer_data_sequence, \
                                            hidden_states, observed_states)
print("\nEmission(output) 결과:")
for state, emissions in emission_matrix.items():
    print(state + ": ", emissions)

Transition 결과:
NORMAL:  {'NORMAL': 0.6013384321223709, 'SEASON OFF': 0.2001160885004097, 'OUTLET': 0.19854547937721934}
SEASON OFF:  {'NORMAL': 0.5942043931112715, 'SEASON OFF': 0.20160874497267195, 'OUTLET': 0.2041868619160565}
OUTLET:  {'NORMAL': 0.5988857938718662, 'SEASON OFF': 0.20251728051170947, 'OUTLET': 0.19859692561642422}

Emission(output) 결과:
NORMAL:  {'1': 0.41597710446089714, '2': 0.33907795682199965, '3': 0.2449449387171032}
SEASON OFF:  {'1': 0.4200810726535703, '2': 0.33121297162457125, '3': 0.24870595572185844}
OUTLET:  {'1': 0.41544163063619516, '2': 0.3330450895614577, '3': 0.2515132798023471}
